In [8]:
# 初始化環境
from rag_to_vector_db_pinecone.rag_pipeline.chunks_embeding_2_vector_db import (
    jina_embedding,
)
from pprint import pprint

import json
from pinecone import Pinecone
from langchain_openai import ChatOpenAI

# 載入 env 設定
with open('env.json', 'r', encoding='utf-8') as file:
    config = json.load(file)

# 初始化 Pinecone gRPC 客戶端
pc = Pinecone(api_key=config['pinecone_api_key'])
index = pc.Index("rag-vector-db")

# 初始化 OpenAI 模型
llm = ChatOpenAI(
    model="gpt-3.5-turbo",  # 可以改為 gpt-3.5-turbo 測試
    openai_api_key=config["chat_gpt_key"],
    temperature=0.6
)

In [2]:
# 使用者輸入查詢 pinecone
def query_pincone(query, config):
    query_embedding = jina_embedding(query, config)
    results = index.query(
        vector=query_embedding,
        top_k=3,
        include_metadata=True
    )

    matches = results['matches']
    # 用 id 匹配資料庫文字
    with open('data/chunks.json', 'r', encoding='utf-8') as file:
        chunks = json.load(file)

    match_chunks_data = []
    for match in matches:
        for chunk in chunks:
            if chunk['id'] == match['id']:
                match_chunks_data.append({
                    "id" : match['id'],
                    "score" : match['score'],
                    "text" : chunk['text']
                })
                break

    return match_chunks_data


def create_augmented_prompt(query, match_chunks_data):
    # 將檢索到的上下文整合成一個字符串
    contexts = ""
    for i, context in enumerate(match_chunks_data):
        contexts += f"參考資料 {i+1} [相關度: {context['score']:.2f}]:\n{context['text']}\n\n"
    
    # 構建結構化提示
    prompt = f"""
    以下是使用者的查詢問題，請根據提供的參考資料回答。如果參考資料中沒有相關資訊，請明確說明。

    使用者查詢: {query}
    請根據以下參考資料提供準確、簡潔且有條理的回答。引用資料時請註明來源編號。

    {contexts}
    """
    
    return prompt


def query_llm(prompt):
    try:
        response = llm.invoke(prompt)
        return response.content
    except:
        return f"LLM 錯誤"

In [10]:
def rag_query(query):
    # 1. 尋找知識庫
    match_chunks_data = query_pincone(query, config)
    print('==========匹配向量資料庫=============')
    pprint(match_chunks_data, sort_dicts=False)

    # 2. 將 知識庫資料結合至 prompt
    prompt = create_augmented_prompt(query, match_chunks_data)
    
    # 3. prompt 引入 LLM
    rag_result = query_llm(prompt)
    
    # 4. rag query
    print('\n\n\n================ RAG result ==========')
    print(rag_result)

In [9]:
rag_query('最具成本效益的家用防霉方法是什麼？')

==========匹配向量資料庫=============
[{'id': 'chunk22',
  'score': 0.687544703,
  'text': '### 6.3 商業防霉產品評估\n'
          '市面上常見防霉產品的效果比較：\n'
          '- 抗菌噴霧：方便使用，適合預防性處理\n'
          '- 氯基漂白劑：殺菌效果好，但可能損傷某些材質\n'
          '- 專業皮革防霉劑：針對皮革設計，通常含防水成分\n'
          '- 天然防霉劑：如茶樹油，對敏感人群更友好'},
 {'id': 'chunk31',
  'score': 0.625386596,
  'text': '### 9.3 家庭防霉的經濟考量\n'
          '消費者層面的防霉經濟學：\n'
          '- 高端鞋類的保養投資與延長壽命收益\n'
          '- 防霉產品的家庭預算分配\n'
          '- 鞋類損失與更換成本分析\n'
          '\n'
          '## 第十章：行業標準與最佳實踐\n'
          '\n'
          '### 10.1 國際標準與規範\n'
          '與鞋類防霉相關的行業標準：\n'
          '- ISO防霉測試標準\n'
          '- 鞋類儲存環境標準\n'
          '- 防霉處理化學品安全規範\n'
          '- 產品防霉性能標準\n'
          '\n'
          '### 10.2 製造商防霉指南\n'
          '鞋類生產商的防霉實踐建議：\n'
          '- 原材料預處理\n'
          '- 生產過程中的防霉控制點\n'
          '- 產品包裝前的處理流程\n'
          '- 質量控制檢查項目'},
 {'id': 'chunk32',
  'score': 0.615278661,
  'text': '### 10.3 零售商與消費者防霉清單\n'
          '實用的防霉檢查清單：\n'
          '- 零售環境日常檢查項目

In [ ]:
# 1. 基礎事實查詢 - 最簡單
rag_query('鞋子發霉怎麼辦？')

==========匹配向量資料庫=============
[{'id': 'chunk16',
  'score': 0.659086585,
  'text': '### 5.2 鞋類保養處理\n'
          '正確的鞋類保養可顯著降低霉變風險：\n'
          '- 使用後處理：確保鞋子完全乾燥後再存放\n'
          '- 定期清潔：移除可能滋生霉菌的污垢和有機物\n'
          '- 專業防護產品：根據鞋材使用適當的防霉劑和防水劑\n'
          '- 吸濕材料：使用矽膠乾燥劑或木炭包吸收多餘水分'},
 {'id': 'chunk1',
  'score': 0.616798699,
  'text': '# 鞋子發霉技術文件\n'
          '\n'
          '第一章：概述與背景\n'
          '第二章：霉菌生長的科學原理\n'
          '第三章：影響鞋類霉變的環境因素\n'
          '第四章：霉菌檢測與鑑定方法\n'
          '第五章：預防措施與控制策略\n'
          '第六章：霉變鞋類處理方法\n'
          '第七章：特殊情境與案例研究\n'
          '第八章：科技與創新\n'
          '第九章：經濟影響與成本分析\n'
          '第十章：行業標準與最佳實踐\n'
          '\n'
          '\n'
          '## 第一章：概述與背景\n'
          '\n'
          '### 1.1 文件目的\n'
          '本技術文件旨在提供有關鞋子發霉問題的全面理解，從霉菌生長的科學原理到預防和處理方法。本文件適用於鞋類製造商、零售商、保養專家及消費者。'},
 {'id': 'chunk18',
  'score': 0.596020222,
  'text': '### 5.3 儲存最佳實踐\n'
          '適當的儲存方式對預防霉變至關重要：\n'
          '- 通風存放：使用開放式鞋架\n'
          '- 定期檢查：尤其是較少穿著的鞋子\n'
          '- 

In [ ]:
# 2. 比較查詢 - 中等複雜度
rag_query('白色和黑色黴菌對鞋子的危害有什麼不同？')

==========匹配向量資料庫=============
[{'id': 'chunk4',
  'score': 0.683558702,
  'text': '### 2.2 常見鞋類霉菌種類\n'
          '- 青黴菌(Penicillium)：呈現藍綠色，常見於皮革鞋\n'
          '- 黑曲黴(Aspergillus niger)：呈現黑色斑點，常見於運動鞋\n'
          '- 毛黴菌(Mucor)：灰白色毛狀，快速生長\n'
          '- 木黴菌(Trichoderma)：綠色，常見於帆布鞋\n'
          '- 根黴菌(Rhizopus)：黑灰色，常侵染鞋內襯'},
 {'id': 'chunk2',
  'score': 0.669094145,
  'text': '### 1.2 霉菌基本概念\n'
          '霉菌是一種真菌類微生物，通過孢子繁殖。在適宜的環境條件下，這些孢子會迅速生長並形成可見的霉菌菌落。霉菌不僅影響鞋子的外觀，還可能導致材料劣化，並對某些人的健康造成影響。\n'
          '\n'
          '### 1.3 鞋子霉變的影響範圍\n'
          '- 美觀問題：霉斑和變色\n'
          '- 材質損壞：纖維分解和結構弱化\n'
          '- 異味產生：特有的霉味\n'
          '- 健康風險：可能引發過敏反應或呼吸問題\n'
          '- 商業損失：庫存損壞與品牌聲譽影響\n'
          '\n'
          '## 第二章：霉菌生長的科學原理'},
 {'id': 'chunk5',
  'score': 0.587453723,
  'text': '### 2.3 不同材質的易感性分析\n'
          '| 鞋類材質 | 霉菌易感性 | 主要風險因素 | 常見霉菌類型 |\n'
          '|---------|----------|------------|------------|\n'
          '| 天然皮革 | 高 | 多孔性，吸濕 | 青黴菌，黑曲黴 |\n'
          '| 帆布 | 高 | 纖維吸收

In [ ]:
# 3. 多步驟處理查詢 - 中高複雜度
rag_query('我的運動鞋裡外都有黴菌，怎麼徹底清洗並防止再次發霉？')

==========匹配向量資料庫=============
[{'id': 'chunk16',
  'score': 0.721057236,
  'text': '### 5.2 鞋類保養處理\n'
          '正確的鞋類保養可顯著降低霉變風險：\n'
          '- 使用後處理：確保鞋子完全乾燥後再存放\n'
          '- 定期清潔：移除可能滋生霉菌的污垢和有機物\n'
          '- 專業防護產品：根據鞋材使用適當的防霉劑和防水劑\n'
          '- 吸濕材料：使用矽膠乾燥劑或木炭包吸收多餘水分'},
 {'id': 'chunk18',
  'score': 0.64476794,
  'text': '### 5.3 儲存最佳實踐\n'
          '適當的儲存方式對預防霉變至關重要：\n'
          '- 通風存放：使用開放式鞋架\n'
          '- 定期檢查：尤其是較少穿著的鞋子\n'
          '- 避免塑料袋：長期存放不使用塑料袋密封\n'
          '- 乾燥劑使用：在鞋盒或儲存區域放置乾燥劑\n'
          '- 防霉產品：考慮使用商業防霉片或香囊\n'
          '\n'
          '#### 5.3.1 長期存放策略\n'
          '長期不穿著的鞋子需要特別注意：\n'
          '1. 徹底清潔和乾燥\n'
          '2. 使用酸性紙包裹（非報紙）\n'
          '3. 放入透氣鞋盒，添加足量乾燥劑\n'
          '4. 存放在乾燥通風處\n'
          '5. 每2-3個月檢查一次\n'
          '6. 定期更換乾燥劑'},
 {'id': 'chunk28',
  'score': 0.61425,
  'text': '#### 7.3.2 高科技運動鞋防霉\n'
          '現代運動鞋中的複合材料需要綜合防護：\n'
          '- 注意鞋內科技材料（如氣墊）不要過度潮濕\n'
          '- 使用不會損害特殊材料的中性清潔劑\n'
          '- 可拆卸部件單獨清潔和乾燥\n

In [14]:
# 4. 專業知識與健康關聯查詢 - 高複雜度
rag_query('長期穿著發霉的鞋子對足部健康有哪些特定風險？特別是對糖尿病患者來說。')

==========匹配向量資料庫=============
[{'id': 'chunk2',
  'score': 0.648993373,
  'text': '### 1.2 霉菌基本概念\n'
          '霉菌是一種真菌類微生物，通過孢子繁殖。在適宜的環境條件下，這些孢子會迅速生長並形成可見的霉菌菌落。霉菌不僅影響鞋子的外觀，還可能導致材料劣化，並對某些人的健康造成影響。\n'
          '\n'
          '### 1.3 鞋子霉變的影響範圍\n'
          '- 美觀問題：霉斑和變色\n'
          '- 材質損壞：纖維分解和結構弱化\n'
          '- 異味產生：特有的霉味\n'
          '- 健康風險：可能引發過敏反應或呼吸問題\n'
          '- 商業損失：庫存損壞與品牌聲譽影響\n'
          '\n'
          '## 第二章：霉菌生長的科學原理'},
 {'id': 'chunk16',
  'score': 0.605824,
  'text': '### 5.2 鞋類保養處理\n'
          '正確的鞋類保養可顯著降低霉變風險：\n'
          '- 使用後處理：確保鞋子完全乾燥後再存放\n'
          '- 定期清潔：移除可能滋生霉菌的污垢和有機物\n'
          '- 專業防護產品：根據鞋材使用適當的防霉劑和防水劑\n'
          '- 吸濕材料：使用矽膠乾燥劑或木炭包吸收多餘水分'},
 {'id': 'chunk6',
  'score': 0.59300375,
  'text': '## 第三章：影響鞋類霉變的環境因素\n'
          '\n'
          '### 3.1 環境濕度的影響\n'
          '相對濕度是鞋類霉變的首要環境因素。當環境相對濕度超過70%時，霉菌生長風險大幅增加。熱帶和亞熱帶地區，尤其是雨季期間，環境濕度經常超過此臨界值。\n'
          '\n'
          '#### 3.1.1 季節性濕度變化\n'
          '不同季節濕度變化對鞋類霉變的影響：\n'
          '- 雨季

In [15]:
# 5. 情境分析與多方案比較查詢 - 最高複雜度
rag_query('我在熱帶高濕地區工作，需要長期在戶外穿著皮靴，已經出現反覆發霉問題。請比較不同防黴方案的效果、成本和實用性，並給出最適合我情況的解決方案。')

==========匹配向量資料庫=============
[{'id': 'chunk27',
  'score': 0.695675492,
  'text': '#### 7.2.2 溫帶季節性潮濕地區\n'
          '案例：日本家庭在梅雨季節的鞋類保養\n'
          '問題：連續數週的高濕度環境導致家庭鞋櫃中鞋類霉變\n'
          '解決方案：\n'
          '- 梅雨季前進行預防性清潔與防護處理\n'
          '- 在鞋櫃中安裝小型除濕器或大量使用除濕盒\n'
          '- 每週定期將鞋子取出通風幾小時\n'
          '- 使用電子防潮鞋架\n'
          '\n'
          '### 7.3 特殊材質鞋類的霉變防護\n'
          '特定鞋類材質需要特別注意：\n'
          '\n'
          '#### 7.3.1 老舊皮革鞋的保養\n'
          '老化皮革對霉菌更為敏感，需要特殊保養：\n'
          '- 定期使用適當的皮革護理油\n'
          '- 避免與塑料接觸存放\n'
          '- 使用皮革專用防霉劑\n'
          '- 考慮使用雪松木鞋楦維持形狀並吸濕'},
 {'id': 'chunk22',
  'score': 0.675851822,
  'text': '### 6.3 商業防霉產品評估\n'
          '市面上常見防霉產品的效果比較：\n'
          '- 抗菌噴霧：方便使用，適合預防性處理\n'
          '- 氯基漂白劑：殺菌效果好，但可能損傷某些材質\n'
          '- 專業皮革防霉劑：針對皮革設計，通常含防水成分\n'
          '- 天然防霉劑：如茶樹油，對敏感人群更友好'},
 {'id': 'chunk28',
  'score': 0.654767811,
  'text': '#### 7.3.2 高科技運動鞋防霉\n'
          '現代運動鞋中的複合材料需要綜合防護：\n'
          '- 注意鞋內科技材料（如氣墊）不要過度潮濕\n'
    